Import Libraries 

In [ ]:
# Importing Libraries
import pandas as pd
import numpy as np
import os

Import Data files

# Creating path 
path = r'C:\Users\Neway\OneDrive\Desktop\instacart Basket Analysis'

In [ ]:
# Importing orders_products_merge_derived.pl file
df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data' , 'Prepared Data' , 'orders_products_merged_derived.pkl'))


In [ ]:
# checking the shape of the dataframe
df_ords_prods_merge.shape


Aggregated mean Calculation for Entire Dataframe 

# Grouping by 'department_id' and 'order_number', and Calculating Mean using agg() Function
df_ords_prods_merge.groupby('department_id').agg({'order_number' : ['mean']})

Comparison Between Subset and Entire Dataframe

General Patterns: The full dataframe shows higher average order numbers across departments overall, suggesting it reflects a wider spectrum of customer activity compared to the subset.

Key Differences: Department 21 stands out in both datasets, but its mean is larger within the subset (25.53) than in the full dataset (22.90). Meanwhile, Department 10 shows a stronger effect in the full dataframe (20.19) versus the subset (18.68).

Stable vs. Variable Results: Certain departments, such as 13 and 17, remain fairly consistent between the two, whereas others—like 3, 9, and 12—display more noticeable differences.

Overall Takeaway: The complete dataframe highlights more frequent ordering behavior across departments, while the subset provides a narrower view with some departments appearing more exaggerated.

Create a Loyalty flag 

In [ ]:
# Aggregating Data with transform() Function
df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [ ]:
C:\Users\Neway\AppData\Local\Temp\ipykernel_18476\3473307984.py:2: FutureWarning: The provided callable <function max at 0x0000019112EFC7C0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [ ]:
# Checking the Output
df_ords_prods_merge.head(15)

In [ ]:
# Creating a 'Loyalty' Label Flag using loc Function
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df_ords_prods_merge.loc[(df_ords_prods_merge['max_order'] <= 40) & (df_ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [ ]:
# Examining Value Counts on 'loyalty_flag'
df_ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

In [ ]:
loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [ ]:
# Checking the Updated Output
df_ords_prods_merge.head(15)

Analyze Spending Habits by Loyalty Category 

In [ ]:
# Determining Basic Statistics of Spending Habits by Loyalty Category
df_ords_prods_merge.groupby('loyalty_flag').agg({'prices' : ['mean', 'min', 'max']})

Loyal customers generally buy less expensive products, averaging $10.39 per item. In contrast, new customers spend the most, with an average of $13.29, while regular customers sit between the two at $12.50. Across all groups, the lowest recorded price is $1.00, but the unusually high maximum of $99,999 points to potential outliers or data quality issues.

CREATE A SPENDING FLAG 

In [ ]:
# Aggregating Data with transform() Function
df_ords_prods_merge['avg_product_price'] = df_ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [ ]:
C:\Users\Neway\AppData\Local\Temp\ipykernel_18476\3473307984.py:2: FutureWarning: The provided callable <function max at 0x0000019112EFC7C0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [ ]:
# Checking Relevant Columns 
df_ords_prods_merge[['user_id', 'avg_product_price']].head(15)

In [ ]:
# Creating a 'Spending' Label Flag using loc Function
df_ords_prods_merge.loc[df_ords_prods_merge['avg_product_price'] >= 10, 'spending_flag'] = 'High spender'
df_ords_prods_merge.loc[df_ords_prods_merge['avg_product_price'] < 10, 'spending_flag'] = 'Low spender'

In [ ]:
# Examining Value Count on 'spending_flag' Column
df_ords_prods_merge['spending_flag'].value_counts(dropna = False)

In [ ]:
spending_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

In [ ]:
# Checking Releavent Columns
df_ords_prods_merge[['user_id', 'avg_product_price', 'spending_flag']].head(15)

## Create an Order Frequency Flag

In [ ]:
# Aggregating Data with transform() Function
df_ords_prods_merge['median_days_between_orders'] = df_ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [ ]:
C:\Users\Neway\AppData\Local\Temp\ipykernel_18476\3473307984.py:2: FutureWarning: The provided callable <function max at 0x0000019112EFC7C0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [ ]:
# Checking Relevant Columns 
df_ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_between_orders']].head(15)

In [ ]:
# Creating a 'Order Frequency' Label Flag using loc Function
df_ords_prods_merge.loc[df_ords_prods_merge['median_days_between_orders'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'
df_ords_prods_merge.loc[(df_ords_prods_merge['median_days_between_orders'] <= 20) & (df_ords_prods_merge['median_days_between_orders'] > 10), 'order_frequency_flag'] = 'Regular customer'
df_ords_prods_merge.loc[df_ords_prods_merge['median_days_between_orders'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [ ]:
# Examining Value Counts on 'order_frequency_flag'
df_ords_prods_merge['order_frequency_flag'].value_counts(dropna = False)

In [ ]:
order_frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: count, dtype: int64

In [ ]:
# Checking Relevant Columns of Updated Dataframe
df_ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_between_orders', 'order_frequency_flag']].head(15)

## Export the Dataframe in pickle Format to the "Prepared Data" Folder

In [ ]:
# Exporting the Dataframe
df_ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_grouped.pkl'))